In [1]:
import pandas as pd
from sodapy import Socrata
import matplotlib.pyplot as plt
import yaml

In [14]:
#initial test pull, used docs: https://dev.socrata.com/foundry/data.pa.gov/dc5b-gebx
try:
    with open("config.yaml", "r") as file:
        config = yaml.safe_load(file)
except FileNotFoundError:
    print("Config file not found.")
except yaml.YAMLError as exc:
    print(f"Error in configuration file: {exc}")

client = Socrata("data.pa.gov", config['APP_TOKEN'])

#request timing out, data set is 3 million rows so probably too big going to pull a year at a time. If total rows equals default rows, double total rows and try again until it returns all values for year
def extract_data(start_year, end_year):
    limit_value = 100000
    year_list = range(start_year, end_year+1)
    full_data = pd.Series()
    for year in year_list:
        print(f'proccessing {year}')
        year_extract = client.get("dc5b-gebx", limit=limit_value, crash_year=year)
        print(len(year_extract))
        while len(year_extract) == limit_value:
            limit_value *= 2
            print(f'limit reached, rerunning with {limit_value}')
            year_extract = client.get("dc5b-gebx", limit=limit_value, crash_year=year)
        year_df = pd.DataFrame.from_records(year_extract)
        full_data = pd.concat([full_data, year_df])
        print({f'{year} added to full data'})
    return full_data


In [15]:
test_df = extract_data(2019, 2020)

proccessing 2019
100000
limit reached, rerunning with 200000
{'2019 added to full data'}
proccessing 2020
83958
{'2020 added to full data'}
